## Classify tomato images using CNN

In [1]:
# import necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout,Dense
from keras import backend as K

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse, random, pickle, cv2, os


TRAIN_DIR = "./train"
MODEL_DIR = "./models"
TEST_DIR = "./Nishant"

Using TensorFlow backend.


In [2]:
## initialize parameters
EPOCHS = 50
INIT_LR = 0.001
BATCH_SIZE = 32
IMAGE_DIMS = (128, 128, 3)

### read images

imagePaths = sorted(list(paths.list_images(TRAIN_DIR)))
data = []
labels = []
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
 
    # extract set of class labels from the image path and update the
    # labels list
    l = imagePath.split(os.path.sep)[-2]
    labels.append(l)

#### Pre-process data

In [3]:
## convert into arrays
data = np.array(data, dtype="float")
labels = np.array(labels)

In [4]:
## label encoding
le = LabelEncoder()
labels_int = le.fit_transform(labels)
labels_en = to_categorical(labels_int)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing

trainX, testX, trainY, testY = train_test_split(data,labels_en, test_size=0.2, random_state=42)
 
# construct the image generator for data augmentation
aug = ImageDataGenerator(rescale=1. / 255,rotation_range=25, shear_range=0.2,
                         zoom_range=0.2,horizontal_flip=True)

#### Model architecture

In [5]:
class DeepNet:
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        # initialize the model along with the input shape to be "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.2))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
 
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        
        # FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
 
        # use a *softmax* activation for single-label classification
        # and *sigmoid* activation for multi-label classification
        model.add(Dense(classes))
        model.add(Activation(finalAct))
 
        # return the constructed network architecture
        return model

In [ ]:
model = DeepNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],depth=IMAGE_DIMS[2], classes=11,finalAct="softmax")
 
# initialize the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
 
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
                        validation_data=(testX, testY),steps_per_epoch=len(trainX) // BATCH_SIZE,
                        epochs=EPOCHSdata = np.array(data, dtype="float")
labels = np.array(labels))

# save the model to disk
model.save(os.path.join(MODEL_DIR,"model.h5"))
model.save_weights(os.path.join(MODEL_DIR,'weights.h5'))

#### Plot train loss and accuracy

In [36]:
len(trainX) // BATCH_SIZE

1077

In [38]:
trainX.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("plot")